In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os
import requests
load_dotenv()

True

In [2]:
# Tool Creation
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_rate(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the current conversion factor between base currency and target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/{os.environ["EXCHANGE_RATE_API"]}/pair/{base_currency.upper()}/{target_currency.upper()}"
    response = requests.get(url)
    return response.json()["conversion_rate"]

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given a currency conversion rate this function will calculates target currency value from a give base currency value
    """
    return base_currency_value * conversion_rate

In [3]:
get_conversion_rate.invoke({"base_currency": "usd", "target_currency": "inr"})

85.7501

In [4]:
convert.invoke({"base_currency_value": 50, "conversion_rate": 85.7501})

4287.505

In [5]:
# Binding tools with LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite-preview-06-17",
    api_key=os.environ["GEMINI_API_KEY"]
)
llm_with_tools = llm.bind_tools([get_conversion_rate, convert])

In [16]:
# Tool calling
message = [HumanMessage("What is the conversion rate between usd and inr, and based on that can you convert 1 million usd to inr?")]

In [17]:
message

[HumanMessage(content='What is the conversion rate between usd and inr, and based on that can you convert 1 million usd to inr?', additional_kwargs={}, response_metadata={})]

In [18]:
ai_message = llm_with_tools.invoke(message)

In [19]:
message.append(ai_message)

In [20]:
ai_message.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '21464039-bdb2-4ba3-8b23-bc0892ea2d3f',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 1000000.0},
  'id': '88a39bae-88ca-4a42-9d54-08bb77d4fb09',
  'type': 'tool_call'}]

In [21]:
for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call["name"] == 'get_conversion_rate':
        tool_message1 = get_conversion_rate.invoke(tool_call)
        # fetch conversion rate
        conversion_rate = tool_message1.content
        # append this tool to messages list
        message.append(tool_message1)
    # execute the 2nd tool using conversion rate from the tool 1st
    if tool_call["name"] == "convert":
        # fetech the current argument
        tool_call["args"]["conversion_rate"] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        message.append(tool_message2)

In [22]:
message

[HumanMessage(content='What is the conversion rate between usd and inr, and based on that can you convert 1 million usd to inr?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 1000000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--5c0cd2ab-ba33-49b8-9da7-758f36e1fd1f-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '21464039-bdb2-4ba3-8b23-bc0892ea2d3f', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 1000000.0, 'conversion_rate': '85.7501'}, 'id': '88a39bae-88ca-4a42-9d54-08bb77d4fb09', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 49, 'total_tokens': 198, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='85.7501', name='get_conversion_rat

In [23]:
llm_with_tools.invoke(message).content

'The conversion rate from USD to INR is 85.7501. 1 million USD is equal to 85,750,100 INR.'